In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
430,2024-07-03,Brasil Lbf Feminina,21:00,Ituano F,SESI Araraquara F,1.82,1.90,131.5,1.82,1.88,-3.5,1.95,2.08,8AagihcT,0.549451,0.526316,0.549451,0.531915,0.075766,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.030413,0.022933,0.045620,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
431,2024-07-03,Argentina Liga A,22:00,Instituto de Cordoba,Olimpico,1.48,2.57,157.5,1.80,1.87,-4.5,1.85,3.50,xUVQr7rm,0.675676,0.389105,0.555556,0.534759,0.064781,0.0,0.2,NaN,NaN,143.556,24.168490,0.168356,105.502,19.914309,0.188758,89.428,132.604,87.32,123.84,0.0,0.0,0.0,0.0,0.380616,0.026974,0.436159,-0.18,-0.036,-13.333333,0.792999,0.8,0.007001,0.36,0.072,21.805556,0.624896,0.6,-0.024896
432,2024-07-03,Canadá Cebl,21:00,Winnipeg Sea Bears,Edmonton Stingers,1.85,1.87,182.5,1.85,1.85,-1.5,1.89,1.92,2g004SCp,0.540541,0.534759,0.540541,0.540541,0.075300,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,180.40,181.39,0.0,0.0,0.0,0.0,0.007603,0.000000,0.011136,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
433,2024-07-03,Colômbia Lbp,21:30,Titanes de Barranquilla,Bucaros,1.54,2.37,155.5,1.84,1.86,-4.5,1.94,1.92,jDcRpc9N,0.649351,0.421941,0.543478,0.537634,0.071292,0.0,0.0,NaN,NaN,114.054,33.311619,0.292069,NaN,NaN,NaN,0.000,0.000,97.58,0.00,0.0,0.0,0.0,0.0,0.300204,0.007644,0.007328,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Brasil Lbf Feminina,Ituano F,SESI Araraquara F,131.5,1.82,0.9998,Over
1,22:00,Argentina Liga A,Instituto de Cordoba,Olimpico,157.5,1.80,1.0000,Over
2,21:30,Colômbia Lbp,Titanes de Barranquilla,Bucaros,155.5,1.84,0.7982,Over
